In [9]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from glob import glob
from scipy.linalg import svd
from scipy.signal import butter, filtfilt

In [10]:
# Functions to create tfrecord
def int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def bytes_feature(value):
    """Convierte un valor en una lista de bytes y crea una característica."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# Functions to process data
def butter_lowpass_filter(data, cutoff, fs, order=4):
    nyq = 0.5 * fs 
    normal_cutoff = cutoff / nyq 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

def ssa(signal, window_length):
    N = len(signal)
    X = np.array([signal[i:i + window_length] for i in range(N - window_length + 1)])
    
    U, S, VT = svd(X, full_matrices=False)
    
    return U, S, VT

In [12]:
# Creating features in a dictionary, accepts an array with shape (6, 500) = (channels, samples)

def window_example(window, prediction, name):
    feature = {
        'reconstructed_signal': float_feature(window[0, :].flatten()),
        'saa1': float_feature(window[1, :].flatten()),
        'saa2': float_feature(window[2, :].flatten()),
        'saa3': float_feature(window[3, :].flatten()),
        'saa4': float_feature(window[4, :].flatten()),
        'saa5': float_feature(window[5, :].flatten()),
        'pred': float_feature(prediction.flatten()),
        'name': bytes_feature(name.encode('utf-8')),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

In [23]:
def create_tfrecord(csv_dir, tfrecord_path):

    csvs = sorted(glob(f"{csv_dir}/*.csv"))
    with tf.io.TFRecordWriter(tfrecord_path) as writer:

        for csv in tqdm(csvs, desc="Leyendo csvs"):

            file = os.path.splitext(os.path.basename(csv))[0]

            # Reading csv signal
            data_file = f'./data/lunar/training/data/S12_GradeA/{file}.csv'
            data_cat = pd.read_csv(data_file)

            # Reading Catalog
            cat_lunar_train_dir = './data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv'
            cat_lunar_train = pd.read_csv(cat_lunar_train_dir)

            detection = cat_lunar_train[cat_lunar_train['filename'] == file].iloc[0]['time_rel(sec)']

            # Columns "time_rel(sec)" y "velocity(m/s)"
            csv_times = np.array(data_cat['time_rel(sec)'].tolist())
            csv_data = np.array(data_cat['velocity(m/s)'].tolist())

            # Filter parameters
            highcut = 0.9  # Frecuencia de corte para el filtro pasa-bajo
            sampling_rate = 1 / 0.15  # Frecuencia de muestreo, derivada de 1 muestra cada 0.15 segundos

            csv_data = butter_lowpass_filter(csv_data, cutoff=highcut, fs=sampling_rate)

            closest_index = np.argmin(np.abs(csv_times - detection))

            # Creating wide window for data labeled with 799 samples
            start_index = max(0, closest_index - 399)
            end_index = min(len(csv_data), closest_index + 399)

            window_data = csv_data[start_index:end_index + 1]

            remaining_data = np.concatenate([csv_data[:start_index], csv_data[end_index + 1:]])

            # windows parameters: for slidding window
            window_size = 500
            stride = 1

            # Number of windows
            n_windows = len(window_data) - window_size + 1

            windows = np.zeros((n_windows, window_size))

            # Generating windows
            for i in range(n_windows):
                windows[i] = window_data[i:i + window_size]
            
            detection_index = 399
            detection_positions = np.zeros(n_windows)

            # Position of detection in each window
            for i in range(n_windows):
                detection_positions[i] = detection_index - i
            
            with_6_seismic = []
            for i in range(len(windows)):

                # Compute SSA 
                window_length = 75
                U, S, VT = ssa(windows[i], window_length)

                # Reconstruction with first 20 components
                n_components = 20 
                reconstructed_trajectory_matrix = np.dot(U[:, :n_components], np.diag(S[:n_components])).dot(VT[:n_components, :])

                reconstructed_signal = np.zeros(len(windows[0]))
                counts = np.zeros(len(windows[0]))

                for i in range(window_length):
                    reconstructed_signal[i: i + len(reconstructed_trajectory_matrix[:, i])] += reconstructed_trajectory_matrix[:, i]
                    counts[i: i + len(reconstructed_trajectory_matrix[:, i])] += 1

                reconstructed_signal /= counts

                # Getting singular values by pairs
                valores_singulares = []
                for i in range(0, 10, 2): 
                    avg_U = (U[:, i:i+1] + U[:, i+1:i+2]) / 2
                    avg_S = (S[i:i+1] + S[i+1:i+2]) / 2
                    avg_VT = (VT[i:i+1, :] + VT[i+1:i+2, :]) / 2

                    reconstructed_trajectory_matrix_i = np.dot(avg_U, np.diag(avg_S)).dot(avg_VT)

                    reconstructed_signal_i = np.zeros(len(windows[0]))
                    counts_i = np.zeros(len(windows[0]))

                    for j in range(window_length):
                        reconstructed_signal_i[j: j + len(reconstructed_trajectory_matrix_i[:, j])] += reconstructed_trajectory_matrix_i[:, j]
                        counts_i[j: j + len(reconstructed_trajectory_matrix_i[:, j])] += 1

                    reconstructed_signal_i /= counts_i

                    valores_singulares.append(reconstructed_signal_i)

                valores_singulares = np.array(valores_singulares)

                # Combining reconstructed and singular values signals
                combined_array = np.zeros((6, 500))

                combined_array[0, :] = reconstructed_signal

                combined_array[1:, :] = valores_singulares

                with_6_seismic.append(combined_array)

            with_6_seismic = np.array(with_6_seismic)

            # Doing the same for data with no seismic detection

            # Parameters
            window_size = 500
            num_windows = 500
            remaining_data_length = len(remaining_data)

            max_start_index = remaining_data_length - window_size

            start_indices = np.random.choice(max_start_index, size=num_windows, replace=False)

            windows_no_label = np.array([remaining_data[start:start + window_size] for start in start_indices])
            ###############################
            no_detection = np.full(500, -1)
            ###############################

            with_6_noise = []
            for i in range(len(windows_no_label)):

                window_length = 75
                U, S, VT = ssa(windows_no_label[i], window_length)

                n_components = 20
                reconstructed_trajectory_matrix = np.dot(U[:, :n_components], np.diag(S[:n_components])).dot(VT[:n_components, :])

                reconstructed_signal = np.zeros(len(windows_no_label[0]))
                counts = np.zeros(len(windows_no_label[0]))

                for i in range(window_length):
                    reconstructed_signal[i: i + len(reconstructed_trajectory_matrix[:, i])] += reconstructed_trajectory_matrix[:, i]
                    counts[i: i + len(reconstructed_trajectory_matrix[:, i])] += 1

                reconstructed_signal /= counts

                valores_singulares = []
                for i in range(0, 10, 2):
                    avg_U = (U[:, i:i+1] + U[:, i+1:i+2]) / 2
                    avg_S = (S[i:i+1] + S[i+1:i+2]) / 2
                    avg_VT = (VT[i:i+1, :] + VT[i+1:i+2, :]) / 2

                    reconstructed_trajectory_matrix_i = np.dot(avg_U, np.diag(avg_S)).dot(avg_VT)

                    reconstructed_signal_i = np.zeros(len(windows_no_label[0]))
                    counts_i = np.zeros(len(windows_no_label[0]))

                    for j in range(window_length):
                        reconstructed_signal_i[j: j + len(reconstructed_trajectory_matrix_i[:, j])] += reconstructed_trajectory_matrix_i[:, j]
                        counts_i[j: j + len(reconstructed_trajectory_matrix_i[:, j])] += 1

                    reconstructed_signal_i /= counts_i

                    valores_singulares.append(reconstructed_signal_i)

                valores_singulares = np.array(valores_singulares)

                combined_array = np.zeros((6, 500))

                combined_array[0, :] = reconstructed_signal

                combined_array[1:, :] = valores_singulares

                with_6_noise.append(combined_array)

            with_6_noise = np.array(with_6_noise)

            # Concatenign data labeled an no labeled
            
            combinated_6_signals = np.concatenate([with_6_seismic, with_6_noise], axis=0) 
            combinated_preds = np.concatenate([detection_positions, no_detection] , axis=0)

            # Normalizing based in exploring_data.ipyb results

            min_val = -2e-8
            max_val = 2e-8

            # Normalización between -1 y 1
            combinated_6_signals_normalized = 2 * (combinated_6_signals - min_val) / (max_val - min_val) - 1

            combinated_preds_normalized = np.where(combinated_preds == -1, -1, combinated_preds / 499)

            # To .tfrecord file
            for i in range(len(combinated_6_signals_normalized)):
                example = window_example(combinated_6_signals_normalized[i], combinated_preds_normalized[i], file)
                writer.write(example.SerializeToString())

In [25]:
if __name__ == '__main__':
    train_dir = './data/lunar/training/data/S12_GradeA/'
    tfrecord_train = './data/lunar/train_ds.tfrecords'

    create_tfrecord(train_dir, tfrecord_train)
    print("====================")


Leyendo csvs: 100%|██████████| 75/75 [21:05<00:00, 16.88s/it]